In [5]:
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from Xception_model import xception
from EFlowersDataset import FlowerDataset
import pandas as pd
import os

In [6]:
class OxfordFlowers(Dataset):

    def __init__(self,excel_file,root_dir,transform=None):

        self.annotations=pd.read_csv(excel_file,delimiter=' ')

        self.root_dir=root_dir

        self.transform=transform

    def __len__(self):

        return len(self.annotations)

    def __getitem__(self,index):

        img_path=os.path.join(self.root_dir,self.annotations.iloc[index,0])

        image=io.imread(img_path)

        y_label=torch.tensor(self.annotations.iloc[index,1])

        image=Image.open(img_path).resize((300,300),resample=0)

        if self.transform:

            image=self.transform(image)

        return (image,y_label)

In [10]:
# os.path.join('./data/processed', 'train/')

training_data = OxfordFlowers(excel_file='./data/oxford-102-flowers/train.txt',root_dir='./data/oxford-102-flowers',  transform=ToTensor())
test_data = OxfordFlowers(excel_file='./data/oxford-102-flowers/valid.txt',root_dir='./data/oxford-102-flowers',  transform=ToTensor())
# training_data = FlowerDataset('./data/processed', transform=ToTensor())
# test_data = FlowerDataset('./data/processed', train=False, transform=ToTensor())
# test_data = datasets.Flowers102(
#     root="data",
#     download=True,
#     transform=ToTensor()
# )


In [11]:
train_dataloader = DataLoader(training_data, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=True)


In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu")))
        y = y.squeeze(0)
        loss = loss_fn(pred, y.to(torch.device("cuda:0")))
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        # print(f'loss: {loss.item()} [{batch*len(X)}/{len(dataloader.dataset)}]')

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu")))
            y = y.squeeze(0).to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [9]:
# print(next(iter(train_dataloader)))

In [10]:
epochs = 150
learning_rate = 0.01
model = xception(num_classes=5, pretrained="imagenet")
model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [11]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 33.7%, Avg loss: 2.258273 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 4.7%, Avg loss: 2.909986 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 4.7%, Avg loss: 2.413450 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 26.7%, Avg loss: 2.107899 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 19.8%, Avg loss: 1.629966 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 15.1%, Avg loss: 2.134204 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 18.6%, Avg loss: 1.621511 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 33.7%, Avg loss: 1.516101 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 43.0%, Avg loss: 1.590317 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 46.5%, Avg loss: 1.322893 

Epoch 11
-------------------------------
Test Error: 
 Accuracy: 25.6%, Avg loss:

KeyboardInterrupt: 

In [12]:
torch.save(model.state_dict(), 'xception_e_flower.pth')